# artists.ipynb
get alphabetized list of artists appearing on a specified playlist
<br>
## Todo
- find a way to append new artists to correct category and label as new

In [17]:
# setup 
import spotipy
import spotipy.util as util
import os
import sys
import pprint
import string
import traceback
import sqlite3

In [22]:
# db connection
conn = sqlite3.connect('spotipy_db.db')
cursor = conn.cursor()

## file setup

In [5]:
## ask for basename - make sure the file is in the working directory of this script
basename = input("name of artist file: ")
## find file
fullname = os.getcwd() + "/save/" + basename + ".txt"

## do checks
if os.path.exists(fullname) and os.path.isfile(fullname):
    artists = set([])
    with open(fullname) as fp:  
        line = fp.readline()
        while line:
            # filter out headers and empty lines
            if line.strip()[:3] != '---' and line.strip()[:3] != '':
                artists.add(line.strip())
            line = fp.readline()
else:
    os.system('touch ' + fullname)
    print("created file '%s'" % fullname)

name of artist file: bangers-artists
--- A ---

AC Slater

Alison Wonderland

Amy Millan





--- B ---

Baauer

Bassnectar

Blanke

Boombox Cartel

Buku





--- C ---

Carmada

Collarbones





--- D ---

DROELOE

Dawn Golden

Deathpact

Delta Heavy

Denzel Curry

Dillon Francis

Diplo





--- E ---

Ekali

Ephwurd





--- F ---

Flosstradamus

Flume

Flux Pavilion

Freshlyground





--- G ---

G-Eazy

Getter

Ghastly

Gravez

graves





--- H ---

Hermitude

Herobust





--- I ---

Ian Everson

Illenium

Infuze





--- J ---

Jack Ü

James Vincent McMorrow

Jauz

Jinzo

Just A Gent





--- K ---

KLP

KRANE

Kai

Kaleem Taylor

Kendrick Lamar

Kerli

Kotek

k?d





--- L ---

LPX

Luca Lush

Luv Drunk





--- M ---

MAX

MYRNE

Mat Zo

Matthew Koma

Miguel

Mr. Carmack

Mt. Eden





--- N ---

NERO

NGHTMRE

Nessly

Nevve





--- O ---





--- P ---

Petit Biscuit

Porter Robinson

Pusha T





--- Q ---

QUIX





--- R ---

RL Grime

Rezz

Ronny J

Rustie





--- S --

In [ ]:
artist_dict = dict.fromkeys(string.ascii_uppercase, [])
artist_dict['#'] = []

for artist in artists:
    first_letter = artist[:1].upper()
    # first letter is not a letter, so bunch into misc. '#' key
    if(first_letter not in artist_dict.keys()):
        artist_dict['#'].append(artist)
    else:
        if artist_dict[first_letter] == []:
            artist_dict[first_letter] = [artist]
        else:
            artist_dict[first_letter].append(artist)

# sort and write to file
with open(basename + '.txt', 'w') as fhandle:
    for key in artist_dict.keys():
        fhandle.write("--- " + key + " ---\n")
        # write artists
        for artist in artist_dict[key] :
            fhandle.write(artist + "\n")
        fhandle.write("\n\n")

In [7]:
## API info
# get this info here: https://developer.spotify.com/dashboard/applications
SPOTIPY_CLIENT_ID     = 'da0aa5c92e024de3827e9db0ea2019fd'
SPOTIPY_CLIENT_SECRET = '8af549cae6224233b7609aea4963b641'
SPOTIPY_REDIRECT_URI  = 'https://github.com/danielpersonius'

# set environment variables - this may or may not be necessary
os.environ['SPOTIPY_CLIENT_ID']=SPOTIPY_CLIENT_ID
os.environ['SPOTIPY_CLIENT_SECRET']=SPOTIPY_CLIENT_SECRET
os.environ['SPOTIPY_REDIRECT_URI']=SPOTIPY_REDIRECT_URI
my_username = "my-username"

# set permissions scope
my_scope = 'user-library-read playlist-read-collaborative playlist-read-private playlist-modify-private playlist-modify-public user-read-playback-state'

# set up request token
token = util.prompt_for_user_token(
        username=my_username,
        scope=my_scope,
        client_id=SPOTIPY_CLIENT_ID,
        client_secret=SPOTIPY_CLIENT_SECRET,
        redirect_uri=SPOTIPY_REDIRECT_URI)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=da0aa5c92e024de3827e9db0ea2019fd&response_type=code&redirect_uri=https%3A%2F%2Fgithub.com%2Fdanielpersonius&scope=playlist-modify-private+playlist-modify-public+playlist-read-collaborative+playlist-read-private+user-library-read+user-read-playback-state in your browser


Enter the URL you were redirected to: https://github.com/danielpersonius?code=AQBa9Si9F8o3MRrmwFtY1cc4QwAwcbES4ecomHxpHGMwhAoIONaKNOjQR17ldDv2lPgq911hNt7TmTIwwQ1aBo7jkMtLsZNdZXhWVZ9MzbcJnCZv8TWBsa-Boqo7nq0johKdgAwxgNEyw9H92_Gzg046vBCb6a5T6tsCLZ0S6TsGPlEhOzTDB3o-LbhoHguIQuTfC7NXpULIkKq1BBNQU0BIfgi1gGTGb4n9Of7fUsZrUgHSxxEIhuePswbmAE52UtYAVz3vFkXQeeFsqarFvo9T_lU-tejOB1qAgYQ_x2Mv-

In [41]:
def get_all_playlists(spot, user_id = None):
    '''
    '''
    if user_id is None:
        results = spot.current_user_playlists()
    else:
        results = spot
    all_playlists = results['items']
    while results['next']:
        results = spot.next(results)
        all_playlists.extend(results['items'])
    return all_playlists

def get_playlist_id(playlist, playlists):
    '''
    given a list of dictionaries of playlist attributes, check if user specified playlist exists
    
    Arguments:
        playlist - string, name of playlist to search for
        playlists - list of possible playlists
    
    Returns:
        int     - index in list of playlists where desired playlist found. -1 if not found 
    '''
    for i in range(0, len(playlists)):
        if playlists[i]['name'] == playlist:
            return playlists[i]['id']
    return None

def get_playlist_tracks(spot, user_id, playlist_id):
    results = spot.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = spot.next(results)
        tracks.extend(results['items'])
    return tracks

def write_artists_to_file(filename: str, artists: dict):
    with open(filename, 'w') as fhandle:
        for key in artists.keys():
            fhandle.write("--- " + key + " ---\n")
            for artist in sorted(artists[key]) :
                fhandle.write(artist + "\n")
            fhandle.write("\n\n")
    print('Done')
    
## get name of desired playlist
playlist = ''
playlist_id = None
# 
index = -1
tracks = []

if token: 
    sp = spotipy.Spotify(auth=token)
    sp.trace = False
    
    user_id = (sp.me())['id']
    all_playlists = get_all_playlists(sp)
    
    # check if playlist exists, reprompt so long as it doesn't
    while playlist_id is None:
        playlist = input("playlist to retrieve artists from: ")
        playlist_id = get_playlist_id(playlist, all_playlists)

    try:
        tracks = get_playlist_tracks(sp, user_id, playlist_id)
        artists = set([])
        for i in range(0, len(tracks)):
            for j in range(0, len(tracks[i]['track']['artists'])):
                #add to db
                artist_id = tracks[i]['track']['artists'][j]['id']
                artist_name = tracks[i]['track']['artists'][j]['name']
                if artist_name != '' and artist_name not in artists:
                    spotify_url = tracks[i]['track']['artists'][j]['external_urls']['spotify']
                    new_artist = (artist_id, artist_name, spotify_url)                    
                    artists.add(artist_name)
                    cursor.execute("INSERT OR IGNORE INTO top_artists(artist_id, artist_name, url) VALUES(?,?,?)", new_artist)
                    conn.commit()
    except Exception as e:
        traceback.print_exc()
        print(e)
else:
    print("Can't get token for: %s" % username)
    raise SystemExit

playlist to retrieve artists from: n_p
